In [1]:
from dotenv import load_dotenv; load_dotenv()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from tqdm import tqdm
import os

from pandarallel import pandarallel
pandarallel.initialize()
from mlb_dataloader.util.data_helpers import sql_read

import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix

from sklearn.metrics.pairwise import cosine_distances

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# ratings = pd.read_csv('ratings.csv').sort_values('reader_id').reset_index(drop=True)
# ratings = sql_read(f'''SELECT * FROM "HAHATON".data_mlrating''')
books = pd.read_csv('books.csv')

/home/jupyter/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ratings = None
m = None
id_to_book = None
reader_to_id = None

In [4]:
def load_ratings():
    global ratings, m, id_to_book, reader_to_id
    if os.getenv('TEST'):
        limit = 100000
    else:
        limit = 999999999
    if ratings is None:
        print('loading')
#         ratings = pd.DataFrame(models.MlRating.objects.values_list(named=True)[:limit])
        ratings = sql_read(f'''SELECT * FROM "HAHATON".data_mlrating''')
        ratings['count'] = 1
        ratings = ratings.groupby(['reader_id', 'doc_id'])['count'].count().reset_index().sort_values('count', ascending=False)

        book_to_id  = {y: x for x, y in enumerate(ratings['doc_id'].sort_values().unique())}
        id_to_book = {y: x for x, y in book_to_id.items()}
        reader_to_id  = {y: x for x, y in enumerate(ratings['reader_id'].sort_values().unique())}
        ratings['doc_id'] = ratings['doc_id'].map(book_to_id)
        ratings['reader_id'] = ratings['reader_id'].map(reader_to_id)
        m = csc_matrix((ratings['count'], (ratings['reader_id'].astype(int).values, 
                                   ratings['doc_id'].astype(int).values)))
        print('loading complete')


In [5]:
def get_top_books(reader_id, n_books=30):
    global m
    if m is None:
        load_ratings()
    dist = (1 - cosine_distances(m, m[reader_to_id[reader_id]])).reshape(-1, )
    w = np.array(list(zip(sorted(dist), np.argsort(dist)))[::-1][:1000])[1:, 0]
    i = np.array(list(zip(sorted(dist), np.argsort(dist)))[::-1][:1000])[1:, 1]
    m2 = m[i].toarray()
    recomended_idxs = [id_to_book[x] for x in np.argsort((m2.T * w).T.sum(axis=0))[::-1][:n_books+1000] if 
                       x not in ratings[ratings.reader_id == reader_to_id[reader_id]]['doc_id'].values]
    return np.array(recomended_idxs[:n_books])



In [6]:
def get_predicts(user_id: int):
    '''
    :param user_id : id from "data.User"
    :return: {'books': [book_1_id, book_2_id, ...],
              'events': [event_1_id, event_2_id, ...],
              'services': [service_1_id, service_2_id, service_3_id, ...]
             }
    '''
    
    books = get_top_books(user_id, n_books=1000).tolist()
    events = []
    services = []
    return {
        'books': books,
        'events': events,
        'services': services,
    }

In [7]:
# def get_top_books(reader_id, n_books=100):
    
#     # reading
#     ratings = pd.read_csv('ratings.csv').sort_values('reader_id').reset_index(drop=True)

#     ratings['count'] = 1
#     ratings = ratings.groupby(['reader_id', 'book_id'])['count'].count().reset_index().sort_values('count', ascending=False)
    
#     book_to_id  = {y: x for x, y in enumerate(ratings['book_id'].sort_values().unique())}
#     id_to_book = {y: x for x, y in book_to_id.items()}
#     reader_to_id  = {y: x for x, y in enumerate(ratings['reader_id'].sort_values().unique())}
#     id_to_reader = {y: x for x, y in reader_to_id.items()}
    
#     ratings['book_id'] = ratings['book_id'].map(book_to_id)
#     ratings['reader_id'] = ratings['reader_id'].map(reader_to_id)
    
#     # Prepare Sparse Matrix# Prepare Sparse Matrix
#     M = csc_matrix((ratings['count'], (ratings['reader_id'].astype(int).values, 
#                                        ratings['book_id'].astype(int).values)))
    
#     # Сalculation
#     dist = (1 - cosine_distances(M, M[reader_to_id[reader_id]])).reshape(-1,)

#     w = np.array(list(zip(sorted(dist), np.argsort(dist)))[::-1][:1000])[1:, 0]
#     i = np.array(list(zip(sorted(dist), np.argsort(dist)))[::-1][:1000])[1:, 1]
    
#     M2 = M[i].toarray()
#     recomended_idxs = [id_to_book[x] for x in np.argsort((M2.T * w).T.sum(axis=0))[::-1][:n_books] if id_to_book[x] not in ratings[ratings.reader_id == reader_id]['book_id'].values]
    
#     return np.array([id_to_book[x] for x in np.argsort((M2.T * w).T.sum(axis=0))[::-1][:n_books]])

In [37]:
reader_id = 332415

In [38]:
# ratings[ratings.reader_id == reader_id]['book_id']

In [39]:
t = get_predicts(reader_id)
t

{'books': [306627,
  6037,
  151562,
  243542,
  11169,
  342730,
  865330,
  1329148,
  706657,
  1234684,
  889749,
  42786,
  5068,
  564880,
  113112,
  375906,
  5476,
  11165,
  3487,
  39958,
  1645087,
  212690,
  128745,
  19085,
  312095,
  826148,
  161785,
  446692,
  272,
  159604,
  1012073,
  371069,
  351376,
  279846,
  1060908,
  951707,
  167205,
  145709,
  3987,
  434412,
  1228583,
  42553,
  5706,
  25243,
  162311,
  549090,
  583141,
  145931,
  765295,
  693852,
  435413,
  206321,
  1151,
  406128,
  587574,
  23100,
  863142,
  756561,
  1015541,
  1034921,
  4883,
  370732,
  1265512,
  945086,
  117820,
  474308,
  257083,
  436319,
  35904,
  42092,
  1458166,
  852340,
  434102,
  1033502,
  127624,
  320082,
  769149,
  1674041,
  736264,
  341216,
  40714,
  1129,
  744590,
  15777,
  713543,
  206069,
  632548,
  197509,
  1488108,
  1234106,
  279359,
  402854,
  721623,
  297081,
  1551761,
  331157,
  11204,
  614540,
  842010,
  1310067,
  79360,


In [40]:
ratings[ratings.reader_id == reader_to_id[reader_id]]['doc_id'].values

array([245235, 244524, 244055, 243751, 243554, 241556, 241049, 240640,
       237268, 236394, 236028, 235979, 235046, 234685, 234207, 234095,
       232295, 232236, 231690, 231663, 231215, 230459, 229175, 229101,
       245041, 245843,  65404, 246530, 260555, 259513, 258039, 257425,
       255160, 254934, 254703, 254616, 254119, 253949, 251776, 250761,
       249741, 249496, 249455, 249153, 249137, 249010, 248801, 247776,
       247573, 247497, 246931, 228378, 227260, 227194, 226245, 206838,
       206693, 206109, 205754, 204798, 204642, 202721, 202660, 202609,
       202263, 201829, 201825, 201732, 201635, 199317, 198935, 198728,
       198712, 197946, 197653, 196929, 196227, 196134, 207718, 209251,
       209409, 219200, 226205, 224341, 223839, 223807, 223355, 222113,
       221747, 220942, 220475, 219379, 219132, 210482, 218787, 218542,
       217732, 217698, 217447, 213742, 213357, 212859, 212075, 211045,
       261525, 263539, 264725, 291778, 310061, 309077, 308654, 306644,
      

In [42]:
books.set_index('id').loc[t['books']].sort_values('rating', ascending=False).head(100).index

Int64Index([ 117820,   11204,   11232,  309052,   11149,   11177,  232215,
             367211,   11224,  260384,   11169, 1555319,    4975,  124653,
             225355,   81285,   11165,   43362,   70766,  121077,  118056,
               3754,   26677,   11188, 1605843,   18285,   26896,  693852,
             300902,   82883,   13765,  282246,  321164,  337293,    3725,
             300929,  253967,  203071,   94295,   86451,    6683,   94819,
              50578,  351133,  785690,    8440,   92749,    1151,   93104,
             357148,  123848,  393981,  388671,   25688,  189998,   81040,
             326275,   16104,     118,  139932,   93118,  539366,  188385,
               5047,  329772,   92829,  341638,  828357,  730225,   16341,
               3162,       1,  270059,   67135,  324401,   42553,  858667,
             306029,  493917,  356095,  246947,  341216,  345279,   12629,
             380820,  238161,  708609,  337252,   94971,  247290,   10759,
              83029,   84

In [29]:
# books.sort_values('rating', ascending=False).head(100)['id'].values

In [74]:
# Дети возраст до 12
if age <= 12:
    np.random.choice([      2,   10668,  117820,   11204,   11232,  309052,   11177,
             272074,   11224,     242,  260384,  562824, 1555319,  231435,
              10693,    4975,   91868,  225355,  309167,  270311,   31895,
              78580,   11145,   72581,   11251,   43362,   70766,  121077,
               1589,   92741,    7373,  430036,   18285,    5923,    6612,
               7247,   11137,  278302,   92753,  152616,  331147,  693852,
             300902,   82883,   11506,  282246,   15204,  337293,    3725,
             300929,  100279,  203071,   76970,   86451,   11247,  306749,
             165773,  202863,    5914,   94819,  351133,  354738,  306686,
              26619,    1346,   92749,    1151,   73815,  357148,  123848,
              29782,  393981,  388671,    2203,  788225,   27972,   26251,
             189998,   81040,   11196,  326275,  331205,     118,  217367,
              93118,  188385,    5047,  144914,   30971,  329772,   92829,
              55409,   78846,  828357, 1632802,  438960,  164933,  730225,
               3534,   67584], n_books, replace=False)
elif age < 19:
# Подростки 13-18
    np.random.choice([      2,  309052,   11157,   11149,     242,  260384,   11169,
             562824,  231435,   10693,    4975,  124653,   91868,  225355,
             117906,  309167,  166748,   31895,   78580,  169332,   72581,
              11165,  118056,   92741,    7373,   18285,    5923,    6612,
               7247,   92753,  152616,  333769,  331147,  693852,   10924,
              11506,   36049,  321164,  337293,   58320,   26315,   76970,
             417745,  165773,  202863,   94819,  351133,  354738,  785690,
             306686,   93034,   26619,    8440,    1151,   73815,    2203,
              25688,   11418,  189998,   93118,  539366,    5047,  329772,
              92829,  276063,   78846,  341638,   16341,   90131,  422557,
             514321,   63110,   30096,  196491,  270059,   15690,  216452,
              67135,   30030,  324401,  265206,   71460,  335149,  426295,
             356095, 1323791,  341216,  299965,  117643,  345279,  238161,
             293855,  337252,   83029,  791537, 1520050,  306627,  366202,
             120733, 1323806], n_books, replace=False)
elif age < 40
# 20-40
    np.random.choice([327018,  279921,  445125,  439155,  309052,   11157,   11149,
             367211,  272074,  365857,   11224,  260384,  327850,  562824,
             231435,   10693,    4975,  118024,  225355,  309167,   81285,
              78580,   72581,   11251,  858521,   43362,  121077,   11188,
             430036,   92753,  333769,  324421,  298538,  693852,   11506,
             321164,  253967,  100279,  118073,   76970,   86451,   11247,
             306749,  351133,    8440,   27255,   93104,  123848,  327716,
             388671,   11418,  189998,  539366,  324804,   75546,    7996,
              20829,   30096,  283837,    3329,   11192,   24074,   22839,
               2259,  324401,  327833,   57777,  858667,  306029,   10766,
               9234,    2034,  117643,    2737,  257274,  380820,   94145,
              93109,  708609,   40690,   10747,  320760,  120733,    4354,
             706560,   11447,    3731, 1117113,   15250,  341268,  326471,
             738145,    6292,  325320,   67165,  251194,   90336,   23610,
             322696,   89747], n_books, replace=False)
else:
# 40+
    np.random.choice([117820,   11204,   11232,  309052,   11149,   11177,  232215,
             367211,   11224,  260384,   11169, 1555319,    4975,  124653,
             225355,   81285,   11165,   43362,   70766,  121077,  118056,
               3754,   26677,   11188, 1605843,   18285,   26896,  693852,
             300902,   82883,   13765,  282246,  321164,  337293,    3725,
             300929,  253967,  203071,   94295,   86451,    6683,   94819,
              50578,  351133,  785690,    8440,   92749,    1151,   93104,
             357148,  123848,  393981,  388671,   25688,  189998,   81040,
             326275,   16104,     118,  139932,   93118,  539366,  188385,
               5047,  329772,   92829,  341638,  828357,  730225,   16341,
               3162,       1,  270059,   67135,  324401,   42553,  858667,
             306029,  493917,  356095,  246947,  341216,  345279,   12629,
             380820,  238161,  708609,  337252,   94971,  247290,   10759,
              83029,   84286,  861373,  187060,  791537,  365301,  306627,
              72495,  366202], n_books, replace=False)

array([  26896,  309052,    5600,   26677,  298538,  365857,  117906,
        321164,  300929,  166748,  117820,       2,  225355,  272074,
         91868,    5923,  270311,  282246,    7247,   92753,   13765,
          4975,    5042,  445125,  169332,    3725,   76970,  309167,
        324421,   11224,   11169,   11251,  304401,    3754,  327018,
         18285,   10924,  439155,   92741,   78580,   36049,   11506,
         13415,   11188,   11228,  100279,  327822,   11145,  124653,
        279921,   10693,   72581,   11137,   26315,  118073,   70766,
        121077,  279900,   31895,    7373,   11216,   81285,  260384,
         58320,   11232,  253967,  693852,   11157, 1605843,  118024,
        562824,  430036,  331147,   15204,    6612,   11149,  232215,
        300902,  327850,  203071,   11177,   11239, 1555319,  333769,
         11204,  417745,  278302,  152616,   11220,  337293,   10668,
        118056,     242,   11165,    1589,   43362,  231435,  858521,
         82883,  367

In [77]:
reader_to_id[42880]

34438